# Match entre empresas y  freelances mediante el índice de Jaccard  

In [18]:
#librerías necesarias
import pandas as pd
import os
import sys
import numpy as np

In [19]:
# El path donde estén tus csv
path="C://Users//lopez//Documents//cursos//master BIG DATA//TFM"
#path="C://Users//Usuario//Desktop//TFM//"
#os.getcwd()
os.chdir(path)
# creamos los dataframes de los csv de compañías y freelances.
# companies = pd.read_excel("companies.xlsx")
# freelances = pd.read_excel("freelances.xlsx")
companies = pd.read_csv("companies.csv")
freelances = pd.read_csv("freelances.csv")

In [20]:
# Cogemos los datasets con los campos que vamos a necesitar
companies_cols=["Company_Name","Industry of company","Prescriptive analytics business","Speech analytics business","Predictive Analytics business",
                "Big Data Business","Machine Learning based business","Highest education","Specialization of highest education",
                "Percent_skill_Entrepreneurship","Percent_skill_Operations","Percent_skill_Engineering",
                "Percent_skill_Leadership", "Percent_skill_Business Strategy"]

companies=companies[companies_cols]

freelances_cols=["Respondent","DeveloperType","FormalEducation","MajorUndergrad","BuildingThings","LearningNewTech","RightWrongWay","CompetePeers",
                "DiversityImportant","ChallengeMyself","ChangeWorld","UnderstandComputers","InvestTimeTools"]

freelances=freelances[freelances_cols]


## Funciones para crear los datasets finales que usaremos para aplicar el íncide de Jaccard

In [21]:
def col_to_dummies(df,col):
    """
    nos quita los NaN, no crea una categoría para ellos
    """
    df_out = pd.get_dummies(df,columns=[col], prefix=col)
    cols=df_out.columns.tolist()[1:]
    df_out[cols]=df_out[cols].astype(float)
    return df_out

#### Funciones relacionadas con habilidades técnicas

In [22]:
# Match entre las columnas de companies:
# Industry of Company
# Predictive Analytics business
# Speech analytics business
# Prescriptive analytics business
# Big Data Business
# Machine Learning based business
# con la de freelance : DeveloperType

def Tech_Skills_Freelancers(freelances,col_Respondent,developer_type):
    """
    Recategorizarión que sale de la col DeveloperType de freelances:
    Software Engineer, Machine Learning, Analytics, Data scientist
    Database or system administrator, Graphics Designer,
    Mobile developer, Web developer, Other
    
    """
    df_dev = freelances[[col_Respondent,developer_type]].copy()
    df_dev[developer_type]=df_dev[developer_type]
    df_dev[developer_type]=df_dev[developer_type].str.replace(';.*','')
    df_dev.loc[(df_dev[developer_type] == "Embedded applications/devices developer") | (df_dev[developer_type] == "Desktop applications developer") |
               (df_dev[developer_type] == "Quality assurance engineer") |(df_dev[developer_type] == "DevOps specialist") ,developer_type] = "Software Engineer"
    df_dev.loc[(df_dev[developer_type] == "Machine learning Specialist"),developer_type] = "Machine Learning"
    #df_dev.loc[(df_dev[developer_type] == "DevOps specialist")  ,developer_type] = "DevOps"
    df_dev.loc[(df_dev[developer_type] == "Developer with a statistics or mathematics background")  ,developer_type] = "Analytics"
    df_dev.loc[(df_dev[developer_type] == "Database administrator") | (df_dev[developer_type] == "Systems administrator") ,developer_type] = "Database or system administrator"
    df_dev.loc[(df_dev[developer_type] == "Graphics programming") | (df_dev[developer_type] == "Graphic designer") ,developer_type] = "Graphics Designer"
    df_out=col_to_dummies(df_dev,developer_type)
    return df_out

def recode_tech_skills_comp(df_companies,col_Company_Name,col_companies):
    """
    Recategorizarión que sale de la col Industry of Company de companies:
    Software Engineer, Analytics, Graphics Designer, Web developer, Mobile developer, other
    Como no existe la categoría Database or system administrator, nos creamos una columna a 0.
    
    """
    df_aux=companies[[col_Company_Name,col_companies]].copy()
    #df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(big data).*','Data scientist')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(analytics|cloud).*','Analytics')
    #df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(machine learning).*','Machine Learning')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(mobile).*','Mobile developer')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(network|marketing|publishing|gaming|advertising).*','Graphics Designer')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(software|cloud).*','software engineer')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(E-commerce).*','Web developer')                                              
    #df_aux[col_companies]=df_aux[col_companies].fillna("sin info")
    df_aux.loc[(df_aux[col_companies] != "Data scientist") & (df_aux[col_companies] != "Analytics") &
               (df_aux[col_companies] != "Machine Learning") & (df_aux[col_companies] != "Mobile developer") &
               (df_aux[col_companies] != "Graphics Designer") & (df_aux[col_companies] != "software engineer") &
               (df_aux[col_companies] != "nan") & (df_aux[col_companies] != "Web developer"),col_companies] = "other"
    df_out=col_to_dummies(df_aux,col_companies)
    df_out["Database or system administrator"]=0.0
    return df_out


cols_companies_DS=["Company_Name","Prescriptive analytics business","Big Data Business","Speech analytics business","Predictive Analytics business"]
cols_companies_ML=["Company_Name","Machine Learning based business"]

def recode_tech_DS_ML_comp(df_companies,cols_companies,out_col_name):
    """
    Recodificamos la columnas asociadas a la categoria de ML o DS como una sóla con valores 0, 1 o nan.
    ds=Prescriptive analytics business,Big Data Business,Speech analytics business,Predictive Analytics business
    ml=Machine Learning based business

    """
    df_aux=companies[cols_companies].copy()
    for i in cols_companies[1:]:
        df_aux[i]=df_aux[i].replace('Yes',1)
        df_aux[i]=df_aux[i].replace('No',0)
        df_aux[i]=df_aux[i].replace('No Info',np.nan)

    df_aux[out_col_name] = df_aux[cols_companies[1:]].max(axis=1, skipna = True)
    df_out=df_aux[[cols_companies[0],out_col_name]]
    return df_out



In [6]:
df_companies1=recode_tech_skills_comp(companies,"Company_Name","Industry of company")
df_companies2=recode_tech_DS_ML_comp(companies,cols_companies_DS,"Data scientist")
df_companies_out=pd.merge(df_companies1,df_companies2,on='Company_Name',how='left')
df_companies3=recode_tech_DS_ML_comp(companies,cols_companies_ML,"Machine Learning")
df_companies_out1=pd.merge(df_companies_out,df_companies3,on='Company_Name',how='left')
df_companies_out1.head()

,Company_Name,Industry of company_Analytics,Industry of company_Graphics Designer,Industry of company_Mobile developer,Industry of company_Web developer,Industry of company_other,Industry of company_software engineer,Database or system administrator,Data scientist,Machine Learning
0,Company1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Company2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,Company3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Company4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Company5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
df_freelance_out1=Tech_Skills_Freelancers(freelances,"Respondent","DeveloperType")
df_freelance_out1.head()

,Respondent,DeveloperType_Analytics,DeveloperType_Data scientist,DeveloperType_Database or system administrator,DeveloperType_Graphics Designer,DeveloperType_Machine learning specialist,DeveloperType_Mobile developer,DeveloperType_Other,DeveloperType_Software Engineer,DeveloperType_Web developer
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


#### Funciones relacionadas con educación

In [23]:
# Match entre la columna de companies Highest education con la de FormalEducation
# Match entre la columna de companies Specialization of highest education con la
# de MajorUndergrad de freelances.

def recode_Highest_education(df_companies,col_Company_Name,col_companies):
    """
    
    """
    df_aux = df_companies[[col_Company_Name,col_companies]].copy()
    df_aux=col_to_dummies(companies[[col_Company_Name,col_companies]],col_companies)
    df_aux["under bachelor's degree"]=0.0
    return df_aux

def recode_FormalEducation(df_freelance,col_Respondent,col_freelance):
    """
    
    """
    df_aux = df_freelance[[col_Respondent,col_freelance]].copy()
    df_aux.loc[df_aux[col_freelance] == "Master's degree",col_freelance] = "Masters"
    df_aux.loc[(df_aux[col_freelance] == "Professional degree") |
               (df_aux[col_freelance] == "Bachelor's degree"),col_freelance] = "Bachelors"
    df_aux.loc[df_aux[col_freelance] == "Doctoral degree",col_freelance] = "PhD"
    df_aux.loc[df_aux[col_freelance] == "I prefer not to answer",col_freelance] = "No Info"
    df_aux.loc[(df_aux[col_freelance] == "Secondary school") |
              (df_aux[col_freelance] == "Some college/university study without earning a bachelor's degree") |
              (df_aux[col_freelance] == "Primary/elementary school") |
              (df_aux[col_freelance] == "I never completed any formal education")
               ,col_freelance] = "under bachelor's degree"
    df_out=col_to_dummies(df_aux,col_freelance)
    return df_out


def recode_MajorUndergrad(df_freelance,col_Respondent,col_freelance):
    """
    
    """
    df_aux = df_freelance[[col_Respondent,col_freelance]].copy()
    df_aux.loc[(df_aux[col_freelance] == "A social science") | (df_aux[col_freelance] == "A business discipline") |
               (df_aux[col_freelance] == "Management information systems") | (df_aux[col_freelance] == "Psychology") |
               (df_aux[col_freelance] == "Information technology, networking, or system administration") |
               (df_aux[col_freelance] == "A humanities discipline"),col_freelance] = "social science"
    df_aux.loc[(df_aux[col_freelance] == "Computer science or software engineering") |
               (df_aux[col_freelance] == "Computer programming or Web development") |
               (df_aux[col_freelance] == "Computer engineering or electrical/electronics engineering") |
               (df_aux[col_freelance] == "Mathematics or statistics"),col_freelance] = "engineering,physics, mathematics or statistics"
    df_aux.loc[df_aux[col_freelance] == "A natural science",col_freelance] = "natural science"
    df_aux.loc[df_aux[col_freelance] == "A health science",col_freelance] = "health science"
    df_aux.loc[df_aux[col_freelance] == "Fine arts or performing arts",col_freelance] = "arts"
    df_aux.loc[(df_aux[col_freelance] == "I never declared a major") |(df_aux[col_freelance] == "Something else") |
              (df_aux[col_freelance] == "A non-computer-focused engineering discipline"),col_freelance] = "other"
    df_aux[col_freelance]=df_aux[col_freelance].fillna("sin info")
    df_out=col_to_dummies(df_aux,col_freelance)
    return df_out

def recode_Specialization_education(df_companies,col_Company_Name,col_companies):
    """
    
    """
    df_aux = df_companies[[col_Company_Name,col_companies]].copy()
    df_aux[col_companies]=df_aux[col_companies]
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(POL.SC|bussiness|Entertainment|human|psychology|journalism|marketing|markting|communic|finance|law|information|economics|Communication|business|politic|nutrition|sociology|dsign|design|history|media|management).*','social science')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(aerospace|artificial intelligence|mining|technology|eng|Computer|math|physics).*','engineering, physics, mathematics or statistics')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(geology|Earth|Biology).*','natural science')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(Biomedical|medicine|nurs).*','health science')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(arts|music|theatre).*','arts')
    df_aux[col_companies]=df_aux[col_companies].fillna("sin info")
    df_aux.loc[(df_aux[col_companies] != "arts") & (df_aux[col_companies] != "health science") &
               (df_aux[col_companies] != "natural science") & (df_aux[col_companies] != "sin info") &
               (df_aux[col_companies] != "social science") &
               (df_aux[col_companies] != "engineering, physics, mathematics or statistics"),col_companies] = "other"
    df_out=col_to_dummies(df_aux,col_companies)
    return df_out

In [ ]:
df_companies_out2=recode_Highest_education(companies,"Company_Name","Highest education")
df_companies_out2.head()

In [ ]:
df_freelance_out2=recode_FormalEducation(freelances,"Respondent","FormalEducation")
df_freelance_out2.head()

In [ ]:
df_companies_out3=recode_Specialization_education(companies,"Company_Name","Specialization of highest education")
df_companies_out3.head()

In [ ]:
df_freelance_out3=recode_MajorUndergrad(freelances,"Respondent","MajorUndergrad")
df_freelance_out3.head()

In [9]:
df_companies1=recode_tech_skills_comp(companies,"Company_Name","Industry of company")
df_companies2=recode_tech_DS_ML_comp(companies,cols_companies_DS,"Data scientist")
df_companies_aux=pd.merge(df_companies1,df_companies2,on='Company_Name',how='left')
df_companies3=recode_tech_DS_ML_comp(companies,cols_companies_ML,"Machine Learning")
df_companies_out1=pd.merge(df_companies_aux,df_companies3,on='Company_Name',how='left')
df_companies_out2=recode_Highest_education(companies,"Company_Name","Highest education")
df_companies_out=pd.merge(df_companies_out1,df_companies_out2,on='Company_Name',how='left')
df_companies_out3=recode_Specialization_education(companies,"Company_Name","Specialization of highest education")
df_companies_out=pd.merge(df_companies_out2,df_companies_out3,on='Company_Name',how='left')
df_companies_out.head()

,Company_Name,Highest education_Bachelors,Highest education_Masters,Highest education_No Info,Highest education_PhD,under bachelor's degree,Specialization of highest education_arts,"Specialization of highest education_engineering, physics, mathematics or statistics",Specialization of highest education_health science,Specialization of highest education_natural science,Specialization of highest education_other,Specialization of highest education_sin info,Specialization of highest education_social science
0,Company1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Company2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Company3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Company4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,Company5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df_freelance_out1=Tech_Skills_Freelancers(freelances,"Respondent","DeveloperType")
df_freelance_out2=recode_FormalEducation(freelances,"Respondent","FormalEducation")
df_freelance_out=pd.merge(df_freelance_out1,df_freelance_out2,on='Respondent',how='left')
df_freelance_out3=recode_MajorUndergrad(freelances,"Respondent","MajorUndergrad")
df_freelance_out=pd.merge(df_freelance_out2,df_freelance_out3,on='Respondent',how='left')
df_freelance_out.head()


,Respondent,FormalEducation_Bachelors,FormalEducation_Masters,FormalEducation_No Info,FormalEducation_PhD,FormalEducation_under bachelor's degree,MajorUndergrad_arts,"MajorUndergrad_engineering,physics, mathematics or statistics",MajorUndergrad_health science,MajorUndergrad_natural science,MajorUndergrad_other,MajorUndergrad_sin info,MajorUndergrad_social science
0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


#### Funciones relacionadas con habilidades

In [24]:
# Falta definir bien los input y output en las funciones.
def recode(companies,col_percent_skill):
    """
    
    """
    df_comp=companies.copy()
    df_comp[col_percent_skill]=df_comp[col_percent_skill].replace('No Info',101)
    df_comp[col_percent_skill]=df_comp[col_percent_skill].astype(float)
    df_comp.loc[(df_comp[col_percent_skill] >=0 ) & (df_comp[col_percent_skill] < 20), col_percent_skill] = 0.2
    df_comp.loc[(df_comp[col_percent_skill] >=20 ) & (df_comp[col_percent_skill] < 40), col_percent_skill] = 0.4
    df_comp.loc[(df_comp[col_percent_skill] >=40) & (df_comp[col_percent_skill] < 60), col_percent_skill] = 0.6
    df_comp.loc[(df_comp[col_percent_skill] >=60 ) & (df_comp[col_percent_skill] < 80), col_percent_skill] = 0.8
    df_comp.loc[(df_comp[col_percent_skill] >=80) & (df_comp[col_percent_skill] < 100), col_percent_skill] = 1
    df_comp.loc[(df_comp[col_percent_skill] == 101.0 ) , col_percent_skill] = "NaN"
    
    return df_comp

def recode_other_skills(df_freelance,col_freelance):
    """
    
    """
    df_aux = df_freelance.copy()
    df_aux.loc[df_aux[col_freelance] == "Strongly disagree",col_freelance] = 0.2
    df_aux.loc[df_aux[col_freelance] == "Disagree",col_freelance] = 0.4
    df_aux.loc[df_aux[col_freelance] == "Somewhat agree",col_freelance] = 0.6
    df_aux.loc[df_aux[col_freelance] == "Agree",col_freelance] = 0.8
    df_aux.loc[df_aux[col_freelance] == "Strongly agree",col_freelance] = 1
    
    return df_aux

In [25]:
def Skill_leadership(freelances_dataset):
    freelances_dataset['Skill_leadership'] = freelances_dataset[["RightWrongWay", "CompetePeers","DiversityImportant","ChallengeMyself"]].sum(axis=1, skipna = True)
    freelances_dataset['Percent_Skill_leadership']=freelances_dataset['Skill_leadership']/4
    freelances_dataset=freelances_dataset.drop(["RightWrongWay", "CompetePeers","DiversityImportant","ChallengeMyself","Skill_leadership"], axis=1)
    return freelances_dataset

def Skill_Operations(freelances_dataset):
    freelances_dataset['Skill_Operations'] = freelances_dataset[["UnderstandComputers", "InvestTimeTools"]].sum(axis=1, skipna = True)
    freelances_dataset['Percent_Skill_Operations']=freelances_dataset['Skill_Operations']/2
    freelances_dataset=freelances_dataset.drop(["UnderstandComputers", "InvestTimeTools","Skill_Operations"],axis=1)
    return freelances_dataset


In [26]:
var_comp_recode=["Company_Name","Percent_skill_Business Strategy","Percent_skill_Entrepreneurship","Percent_skill_Operations","Percent_skill_Leadership","Percent_skill_Engineering"]
var_freelances_recode=["Respondent","BuildingThings","LearningNewTech", "RightWrongWay","CompetePeers", "DiversityImportant","ChallengeMyself","UnderstandComputers","InvestTimeTools","ChangeWorld"]

def skills_comp(companies,var_comp_recode):
    companies_out=companies.copy()
    for i in var_comp_recode[1:]:
        companies_out = recode(companies_out[var_comp_recode],i)
    return companies_out
    
def skills_freel(freelances,var_freelances_recode):
    freelances_out=freelances[var_freelances_recode].copy()
    for x in var_freelances_recode[1:]:
        freelances_out = recode_other_skills(freelances_out[var_freelances_recode],x)
    freelances_out=Skill_leadership(freelances_out)    
    freelances_out=Skill_Operations(freelances_out)    
    return freelances_out


In [ ]:
df_freelance_out4=skills_freel(freelances,var_freelances_recode)
df_freelance_out4.head()

In [ ]:
df_companies_out4=skills_comp(companies,var_comp_recode)
df_companies_out4.head()

In [ ]:
### ahora unificamos todo

In [27]:
df_freelance_out1=Tech_Skills_Freelancers(freelances,"Respondent","DeveloperType")
df_freelance_out2=recode_FormalEducation(freelances,"Respondent","FormalEducation")
df_freelance_out=pd.merge(df_freelance_out1,df_freelance_out2,on='Respondent',how='left')
df_freelance_out3=recode_MajorUndergrad(freelances,"Respondent","MajorUndergrad")
df_freelance_out=pd.merge(df_freelance_out,df_freelance_out3,on='Respondent',how='left')
df_freelance_out4=skills_freel(freelances,var_freelances_recode)
df_freelance_out=pd.merge(df_freelance_out,df_freelance_out4,on='Respondent',how='left')
freelances_dataset=df_freelance_out
df_freelance_out.head()

,Respondent,DeveloperType_Analytics,DeveloperType_Data scientist,DeveloperType_Database or system administrator,DeveloperType_Graphics Designer,DeveloperType_Machine learning specialist,DeveloperType_Mobile developer,DeveloperType_Other,DeveloperType_Software Engineer,DeveloperType_Web developer,...,MajorUndergrad_health science,MajorUndergrad_natural science,MajorUndergrad_other,MajorUndergrad_sin info,MajorUndergrad_social science,BuildingThings,LearningNewTech,ChangeWorld,Percent_Skill_leadership,Percent_Skill_Operations
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1,0.8,0.8,0.65,0.7
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.00,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,1,0.8,0.65,0.5
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1,1,0.8,0.75,0.5
4,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.00,0.0


In [28]:
df_companies1=recode_tech_skills_comp(companies,"Company_Name","Industry of company")
df_companies2=recode_tech_DS_ML_comp(companies,cols_companies_DS,"Data scientist")
df_companies_aux=pd.merge(df_companies1,df_companies2,on='Company_Name',how='left')
df_companies3=recode_tech_DS_ML_comp(companies,cols_companies_ML,"Machine Learning")
df_companies_out1=pd.merge(df_companies_aux,df_companies3,on='Company_Name',how='left')
df_companies_out2=recode_Highest_education(companies,"Company_Name","Highest education")
df_companies_out=pd.merge(df_companies_out1,df_companies_out2,on='Company_Name',how='left')
df_companies_out3=recode_Specialization_education(companies,"Company_Name","Specialization of highest education")
df_companies_out=pd.merge(df_companies_out,df_companies_out3,on='Company_Name',how='left')
df_companies_out4=skills_comp(companies,var_comp_recode)
df_companies_out=pd.merge(df_companies_out,df_companies_out4,on='Company_Name',how='left')
companies_dataset=df_companies_out
companies_dataset.head()

,Company_Name,Industry of company_Analytics,Industry of company_Graphics Designer,Industry of company_Mobile developer,Industry of company_Web developer,Industry of company_other,Industry of company_software engineer,Database or system administrator,Data scientist,Machine Learning,...,Specialization of highest education_health science,Specialization of highest education_natural science,Specialization of highest education_other,Specialization of highest education_sin info,Specialization of highest education_social science,Percent_skill_Business Strategy,Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Leadership,Percent_skill_Engineering
0,Company1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.2,0.2,0.2,0.2,0.2
1,Company2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.4,0.2,0.2,0.2,0.2
2,Company3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.2,0.2,0.2,0.2,0.6
3,Company4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.2,0.2,0.2
4,Company5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.2,0.2,0.2


In [86]:
companies_dataset.describe()

,Industry of company_Analytics,Industry of company_Graphics Designer,Industry of company_Mobile developer,Industry of company_Web developer,Industry of company_other,Industry of company_software engineer,Database or system administrator,Data scientist,Machine Learning,Highest education_Bachelors,...,Specialization of highest education_health science,Specialization of highest education_natural science,Specialization of highest education_other,Specialization of highest education_sin info,Specialization of highest education_social science,Percent_skill_Business Strategy,Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Leadership,Percent_skill_Engineering
count,472.000000,472.000000,472.000000,472.000000,472.000000,472.000000,472.0,467.000000,466.000000,472.000000,...,472.000000,472.000000,472.000000,472.000000,472.000000,411.000000,411.000000,411.000000,411.000000,411.000000
mean,0.442797,0.093220,0.057203,0.027542,0.355932,0.023305,0.0,0.623126,0.276824,0.358051,...,0.008475,0.008475,0.288136,0.205508,0.275424,0.246229,0.695864,0.202920,0.205353,1.048175
std,0.497244,0.291049,0.232477,0.163831,0.479303,0.151031,0.0,0.485123,0.447910,0.479936,...,0.091764,0.091764,0.453375,0.404501,0.447202,0.091086,6.952779,0.027784,0.035208,8.497731
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.200000,0.200000,0.200000,0.200000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.200000,0.200000,0.200000,0.200000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.200000,0.200000,0.200000,0.200000
75%,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,1.000000,1.000000,1.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,0.200000,0.200000,0.200000,0.200000,0.400000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.600000,100.000000,0.600000,0.600000,100.000000


#### Generamos los dataset finales

In [29]:
freelances_dataset.to_excel("freelances_final.xlsx", index=False)
companies_dataset.to_excel("companies_final.xlsx", index=False)

In [30]:
freelances_dataset.to_csv("freelances_final.csv", index=False)
companies_dataset.to_csv("companies_final.csv", index=False)

In [34]:
freelances_dataset = pd.read_excel("freelances_final.xlsx")
companies_dataset= pd.read_excel("companies_final.xlsx")


In [36]:
freelances_dataset=freelances_dataset2
companies_dataset=companies_dataset2

In [35]:
freelances_dataset1 = pd.read_csv("freelances_final.csv")
companies_dataset1= pd.read_csv("companies_final.csv")

In [56]:
# Aplicación del índice de Jaccard 

from sklearn.metrics import jaccard_score
import numpy as np

# Ejemplo
comp_names=companies_dataset.Company_Name.tolist()
freelances_names=freelances_dataset.Respondent.tolist()
companies=np.array(companies_dataset[1:])
freelances=np.array(freelances_dataset[1:])
#Buscando todas las combinaciones posibles
tuples = []
#compania_t=companias.index.values
lista=[]

for i in range(len(companies)):
    for j in range(len(freelances)):
        comp_x=companies
        free_y=freelances
        jaccard=jaccard_score(comp_x[i], free_y[j],average="weighted")
        lista.append(jaccard)
        tuples.append([comp_names[i],freelances_names[j]])
                      
df_match=pd.DataFrame({'company-freelance' : tuples, 'score':lista})
print(df_match)

TypeError: '<' not supported between instances of 'int' and 'str'

In [77]:
companies_dataset.isnull().sum().sum
companies_dataset.dtypes

Company_Name                                                                            object
Industry of company_Analytics                                                            int64
Industry of company_Graphics Designer                                                    int64
Industry of company_Mobile developer                                                     int64
Industry of company_Web developer                                                        int64
Industry of company_other                                                                int64
Industry of company_software engineer                                                    int64
Database or system administrator                                                         int64
Data scientist                                                                         float64
Machine Learning                                                                       float64
Highest education_Bachelors                       

In [83]:
# Ejemplo
comp_names=companies_dataset.Company_Name.tolist()
freelances_names=freelances_dataset.Respondent.tolist()
companies=np.array([[1,0,0],[2,1,1]])
freelances=np.array([[1,2,1],[1,"1",1],[7,7,7]])
freelances=freelances.astype(float)
#Buscando todas las combinaciones posibles
tuples = []
#compania_t=companias.index.values
lista=[]

for i in range(len(companies)):
    for j in range(len(freelances)):
        comp_x=companies
        free_y=freelances
        jaccard=jaccard_score(comp_x[i], free_y[j],average="weighted")
        lista.append(jaccard)
        tuples.append([comp_names[i],freelances_names[j]])
                      
df_match=pd.DataFrame({'company-freelance' : tuples, 'score':lista})
print(df_match)

  company-freelance     score
0     [Company1, 1]  0.166667
1     [Company1, 2]  0.111111
2     [Company1, 3]  0.000000
3     [Company2, 1]  0.222222
4     [Company2, 2]  0.444444
5     [Company2, 3]  0.000000


In [ ]:
companias= pd.read_excel(open("C://Users//Usuario//Desktop//TFM//Companias.xlsx",'rb'),index_col=0)
freelances= pd.read_excel(open("C://Users//Usuario//Desktop//TFM//Frelancer.xlsx",'rb'), index_col=0)

#Para una fila 
from sklearn.metrics import jaccard_similarity_score
comp=companias.iloc[0]
free=freelances.iloc[0]
jaccard_similarity_score(comp, free)
print()

#Iterando en fila 1 con fila 1, fila 2 con fila 2 

#Buscando todas las combinaciones posibles
import itertools
num = [0,1,2,3,4,5]
tuples = []
compania_t=companias.index.values

res = itertools.combinations(num, 2)
for e in res:
    tuples.append(e)
        

lista=[]

for i in range(len(tuples)):
    x=tuples[i][0]
    y=tuples[i][1]
    comp_x=companias.iloc[x]
    free_y=freelances.iloc[y]
    jaccard=jaccard_similarity_score(comp_x, free_y)
    lista.append(jaccard)
    
df_match=pd.DataFrame({'tuplas': tuples, 'score':lista})
print(df_match)

In [85]:
from sklearn.metrics import jaccard_similarity_score
jaccard=jaccard_similarity_score([2,"NaN",2], [2,1,2])
jaccard

C:\Users\lopez\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


0.0